In [0]:
!pip install pytorch_pretrained_bert
!pip install pyknp 

In [0]:
!wget http://lotus.kuee.kyoto-u.ac.jp/nl-resource/jumanpp/jumanpp-1.02.tar.xz

In [0]:
!tar Jxfv jumanpp-1.02.tar.xz

In [16]:
%cd jumanpp-1.02

/content/jumanpp-1.02


In [0]:
!./configure

In [0]:
!make

In [0]:
!sudo make install

In [32]:
%cd ..

/content


In [0]:
# ref:https://github.com/yagays/pytorch_bert_japanese

from pathlib import Path

import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
from pyknp import Juman


class JumanTokenizer():
    def __init__(self):
        self.juman = Juman()

    def tokenize(self, text):
        result = self.juman.analysis(text)
        return [mrph.midasi for mrph in result.mrph_list()]


class BertWithJumanModel():
    def __init__(self, bert_path, vocab_file_name="vocab.txt", use_cuda=False):
        self.juman_tokenizer = JumanTokenizer()
        self.model = BertModel.from_pretrained(bert_path)
        self.bert_tokenizer = BertTokenizer(Path(bert_path) / vocab_file_name,
                                            do_lower_case=False, do_basic_tokenize=False)
        self.use_cuda = use_cuda

    def _preprocess_text(self, text):
        return text.replace(" ", "")  # for Juman

    def get_sentence_embedding(self, text, pooling_layer=-2, pooling_strategy="REDUCE_MEAN"):
        preprocessed_text = self._preprocess_text(text)
        tokens = self.juman_tokenizer.tokenize(preprocessed_text)
        bert_tokens = self.bert_tokenizer.tokenize(" ".join(tokens))
        ids = self.bert_tokenizer.convert_tokens_to_ids(["[CLS]"] + bert_tokens[:126] + ["[SEP]"]) # max_seq_len-2
        tokens_tensor = torch.tensor(ids).reshape(1, -1)

        if self.use_cuda:
            tokens_tensor = tokens_tensor.to('cuda')
            self.model.to('cuda')

        self.model.eval()
        with torch.no_grad():
            all_encoder_layers, _ = self.model(tokens_tensor)

        embedding = all_encoder_layers[pooling_layer].cpu().numpy()[0]
        if pooling_strategy == "REDUCE_MEAN":
            return np.mean(embedding, axis=0)
        elif pooling_strategy == "REDUCE_MAX":
            return np.max(embedding, axis=0)
        elif pooling_strategy == "REDUCE_MEAN_MAX":
            return np.r_[np.max(embedding, axis=0), np.mean(embedding, axis=0)]
        elif pooling_strategy == "CLS_TOKEN":
            return embedding[0]
        else:
            raise ValueError("specify valid pooling_strategy: {REDUCE_MEAN, REDUCE_MAX, REDUCE_MEAN_MAX, CLS_TOKEN}")

In [7]:
!wget http://nlp.ist.i.kyoto-u.ac.jp/DLcounter/lime.cgi?down=http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE.zip&name=Japanese_L-12_H-768_A-12_E-30_BPE.zip

--2020-01-16 12:08:42--  http://nlp.ist.i.kyoto-u.ac.jp/DLcounter/lime.cgi?down=http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE.zip
Resolving nlp.ist.i.kyoto-u.ac.jp (nlp.ist.i.kyoto-u.ac.jp)... 133.3.252.171
Connecting to nlp.ist.i.kyoto-u.ac.jp (nlp.ist.i.kyoto-u.ac.jp)|133.3.252.171|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE.zip [following]
--2020-01-16 12:08:43--  http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE.zip
Reusing existing connection to nlp.ist.i.kyoto-u.ac.jp:80.
HTTP request sent, awaiting response... 200 OK
Length: 1627355542 (1.5G) [application/zip]
Saving to: ‘lime.cgi?down=http:%2F%2Fnlp.ist.i.kyoto-u.ac.jp%2Fnl-resource%2FJapaneseBertPretrainedModel%2FJapanese_L-12_H-768_A-12_E-30_BPE.zip’

lime.cgi?down=http: 100%[=

In [8]:
ls

'lime.cgi?down=http:%2F%2Fnlp.ist.i.kyoto-u.ac.jp%2Fnl-resource%2FJapaneseBertPretrainedModel%2FJapanese_L-12_H-768_A-12_E-30_BPE.zip'
 sample_data/


In [9]:
!unzip 'lime.cgi?down=http:%2F%2Fnlp.ist.i.kyoto-u.ac.jp%2Fnl-resource%2FJapaneseBertPretrainedModel%2FJapanese_L-12_H-768_A-12_E-30_BPE.zip'

Archive:  lime.cgi?down=http:%2F%2Fnlp.ist.i.kyoto-u.ac.jp%2Fnl-resource%2FJapaneseBertPretrainedModel%2FJapanese_L-12_H-768_A-12_E-30_BPE.zip
  inflating: Japanese_L-12_H-768_A-12_E-30_BPE/README.txt  
  inflating: Japanese_L-12_H-768_A-12_E-30_BPE/bert_config.json  
  inflating: Japanese_L-12_H-768_A-12_E-30_BPE/bert_model.ckpt.data-00000-of-00001  
  inflating: Japanese_L-12_H-768_A-12_E-30_BPE/bert_model.ckpt.index  
  inflating: Japanese_L-12_H-768_A-12_E-30_BPE/bert_model.ckpt.meta  
  inflating: Japanese_L-12_H-768_A-12_E-30_BPE/pytorch_model.bin  
  inflating: Japanese_L-12_H-768_A-12_E-30_BPE/vocab.txt  


In [31]:
ls

aclocal.m4      config.sub*   dict-build/         Makefile      README.md
compile*        configure*    INSTALL*            Makefile.am   sample/
config.guess*   configure.ac  install-sh*         Makefile.in   script/
config.log      COPYING*      jumanpp-manual.pdf  missing*      src/
config.status*  depcomp*      jumanpp-resource/   README_ja.md  VERSION


In [0]:
bert = BertWithJumanModel("./Japanese_L-12_H-768_A-12_E-30_BPE")

In [141]:
bert.get_sentence_embedding("こんにちは")

array([ 4.67476666e-01,  2.79777676e-01,  2.95888126e-01, -9.14181709e-01,
        9.01693702e-01, -4.95860010e-01,  2.11472720e-01,  2.33361963e-02,
        7.12771416e-01,  1.84324998e-02, -7.49753952e-01,  1.17939696e-01,
        8.65387738e-01, -9.99620259e-01, -2.05743164e-01, -1.62905127e-01,
       -8.99593353e-01, -7.32729673e-01,  8.59157965e-02,  1.18994355e-01,
       -3.21010470e-01,  2.71281391e-01, -2.75354385e-01,  9.67957973e-01,
       -1.94003999e-01, -8.63049865e-01,  2.44561285e-01,  4.09338087e-01,
       -1.03395915e+00, -1.72082216e-01, -1.38608173e-01,  5.66869617e-01,
       -3.55134338e-01,  2.90463507e-01,  5.24332762e-01, -1.75024346e-01,
        9.08129036e-01, -2.12269381e-01, -2.25464143e-02, -9.75623250e-01,
       -2.66742527e-01,  1.35401934e-01,  5.61718568e-02, -3.02916497e-01,
        1.49506778e-02, -8.82368684e-01,  3.87708634e-01, -2.05565766e-01,
        9.98200476e-03, -1.92200989e-01, -2.80782133e-01, -1.31892249e-01,
       -4.17766035e-01,  

In [0]:
import os
import time
from typing import List, Dict
import re
from datetime import datetime as dt
import pandas as pd
import numpy as np
import boto3
from boto3.dynamodb.conditions import Key
from fastprogress import progress_bar as pb
import torch
import torch.nn as nn

In [0]:
_aws_session = boto3.session.Session(
    region_name='ap-northeast-1',
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

DYNAMO_DB = _aws_session.resource('dynamodb')

In [0]:
class DynamoDB:

    @staticmethod
    def put_items(
        table_name: str,
        items: List[Dict],
        use_batch_writer: bool = False,
    ) -> List:
        """[summary]
        
        Args:
            table_name (str): [description]
            items (List[Dict]): [description]
        
        Raises:
            e: [description]
        
        Returns:
            List: [description]
        """
        if not isinstance(items, list):
            items = [items]

        responses = []
        try:
            table = DYNAMO_DB.Table(table_name)
        except Exception as e:
            print(f'Failed at DYNAMO_DB.Table(table_name) : {e}')
            raise e

        if use_batch_writer:
            with table.batch_writer() as batch:
                for item in items:
                    try:
                        responses.append(
                            batch.put_item(
                                Item=item,
                            )
                        )
                    except Exception as e:
                        print(f'Failed to put data to DynamoDB. Skipping : {e}')
        else:
            for item in items:
                try:
                    responses.append(
                        table.put_item(
                            TableName=table_name,
                            Item=item,
                        )
                    )
                except Exception as e:
                    print(f'Failed to put data to DynamoDB. Skipping : {e}')

        return responses

    @staticmethod
    def partitionkey_query(
        table_name: str,
        partition_key_name: str,
        partition_key: str,
    ) -> List[Dict]:
        try:
            table = DYNAMO_DB.Table(table_name)
        except Exception as e:
            print(f'Failed at DYNAMO_DB.Table(table_name) : {e}')
            raise e

        try:
            response = table.query(
                KeyConditionExpression=Key(partition_key_name).eq(partition_key)
            )
        except Exception as e:
            print(f'Failed to query : {e}')
            return []

        return response['Items']

In [0]:
res = DynamoDB.partitionkey_query(
    table_name='finapp_twitter_tweet',
    partition_key_name='keyword',
    partition_key='アクセル',
)

In [40]:
res[:2]

[{'contributors': None,
  'coordinates': None,
  'created_at': Decimal('1576093163'),
  'entities': {'hashtags': [{'indices': [Decimal('101'), Decimal('108')],
     'text': '湾岸マキシ5'}],
   'symbols': [],
   'urls': [],
   'user_mentions': []},
  'favorite_count': Decimal('0'),
  'favorited': False,
  'geo': None,
  'id': Decimal('1204984066581262338'),
  'id_str': '1204984066581262338',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'keyword': 'アクセル',
  'lang': 'ja',
  'metadata': {'iso_language_code': 'ja', 'result_type': 'recent'},
  'place': None,
  'retweet_count': Decimal('0'),
  'retweeted': False,
  'source': '<a href="http://twittbot.net/" rel="nofollow">twittbot.net</a>',
  'text': '筐体のシートの前後の位置は、アクセルペダルを踏み込んで調整するときに、ハーフが一番やりやすい位置に合わせるのがよい。\n\n答えは〇\n座席の位置は人によってまちまちです。自分の位置に毎回調整しましょう。\n\n#湾岸マキシ5',
  'truncated': False,
  'user'

In [41]:
len(res)

285

In [0]:
texts = list(set([r['text'] for r in res]))

In [43]:
texts

['アクセル踏むとマッハの悪癖私もあるなあ……',
 'RT @008000dic: ナイス 【錆喰いビスコ4 業花の帝冠、花束の剣 (電撃文庫)/瘤久保 慎司】読んでなかった第二部開始の第四巻。相も変わらずアクセルぶっ放しのコンビだが、敵も味方も愚直な熱い奴らが目白押し。サタハ… → https://t.co/gHnzoRbsRw…',
 'RT @belmond_b_2434: やるわ\n【マリオカート８ DX】はい！ここでアクセルですね教官！【にじさんじ】 https://t.co/REUtDf2BnY @YouTubeさんから',
 'Wﾚ*0ω0)おはよアクセル\u3000\u3000おはよロクサス(；∀； ≧＞',
 'アクセルナットすぐ取れる奴',
 'RT @chipdale591106: えげつい瞬間見てしまった。救急車に突っ込んだ。右上の場所消防署。出動して2分で事故！えげつい！救急車待ってる人もかわいそうだし、救急隊員の人悪くないのに謝ってたし、怪我してないか聞いてたのに、どえらい衝撃でしたからね！とか運転手言ってるし…',
 'RT @riderzio2068: #特撮トリビア\nクロックアップは設定上、１分経つと強制解除される…\n\n…と言うのは。ガセ \n\nクロックアップは、変身者に無理を強いらぬ為に…僅か〝数秒〟でクロックオーバーする。\n\n数十秒ですらも無い故に、一回のクロックアップの持続時間はアクセ…',
 '劣等生読み終わったら、バカテスとアクセルワールド読むことにした',
 'RT @yu___he0703: 損保ジャパンの問題が色々流れてるけど間違いなく入らないほうがいいし相手にしたくない。\n\n3月の事故、相手が確認不足、アクセル踏んだって言ってても損保ジャパンは避けれただろって。弁護士も呆れてるくらいだから。入らない方がいい。 https://t…',
 '@62BKW 俺も最初、アクセルブレーキの繊細さにビックリした。特にアクセル。',
 'RT @ncc170116: @sunafukin99 @hokuetsu_noto もう、維新支持者のテンプレだった「都構想が嫌なら大阪から出て行け」を知事や市長が自ら言いそうな勢いですわ。結局、公明こそが維新のアクセル役だったんですなあ。',
 '@kobamac おはよ

In [0]:
dummy_tweet = '9867 アクセル\n\nストップになる前にちょっと買えたので、ストップ高で売っちゃいます。\n小銭稼ぎ。\nまだ上がりそうな割安ですが、利益率低いのが気になるのでやめときます。'

In [0]:
texts.append(dummy_tweet)

In [0]:
reference_sentence1 = 'アクセルの社員がインサイダー取引により逮捕された。'
reference_sentence2 = 'アクセル、業績好調により上方修正を発表した。'
reference_sentence3 = 'おじいちゃんがアクセルを踏み込みコンビニに突っ込んだ。'

In [0]:
reference_sentence1_vec = bert.get_sentence_embedding(reference_sentence1)
reference_sentence2_vec = bert.get_sentence_embedding(reference_sentence2)
reference_sentence3_vec = bert.get_sentence_embedding(reference_sentence3)

In [138]:
reference_sentence1_vec.shape

(768,)

In [139]:
tweet_vecs = []
fail_cnt = 0

for i, tweet in enumerate(pb(texts)):
    print(tweet)
    tweet = tweet.replace('#', '').replace('@', '')
    try:
        vec = bert.get_sentence_embedding(tweet)
        tweet_vecs.append(vec)
    except Exception as e:
        tweet_vecs.append(None)
        fail_cnt += 1
    # time.sleep(1)
    print('='*50)

アクセル踏むとマッハの悪癖私もあるなあ……
RT @008000dic: ナイス 【錆喰いビスコ4 業花の帝冠、花束の剣 (電撃文庫)/瘤久保 慎司】読んでなかった第二部開始の第四巻。相も変わらずアクセルぶっ放しのコンビだが、敵も味方も愚直な熱い奴らが目白押し。サタハ… → https://t.co/gHnzoRbsRw…
RT @belmond_b_2434: やるわ
【マリオカート８ DX】はい！ここでアクセルですね教官！【にじさんじ】 https://t.co/REUtDf2BnY @YouTubeさんから


Analysis is done ignoring "\n".


Wﾚ*0ω0)おはよアクセル　　おはよロクサス(；∀； ≧＞
アクセルナットすぐ取れる奴
RT @chipdale591106: えげつい瞬間見てしまった。救急車に突っ込んだ。右上の場所消防署。出動して2分で事故！えげつい！救急車待ってる人もかわいそうだし、救急隊員の人悪くないのに謝ってたし、怪我してないか聞いてたのに、どえらい衝撃でしたからね！とか運転手言ってるし…
RT @riderzio2068: #特撮トリビア
クロックアップは設定上、１分経つと強制解除される…

…と言うのは。ガセ 

クロックアップは、変身者に無理を強いらぬ為に…僅か〝数秒〟でクロックオーバーする。

数十秒ですらも無い故に、一回のクロックアップの持続時間はアクセ…


Analysis is done ignoring "\n".


劣等生読み終わったら、バカテスとアクセルワールド読むことにした
RT @yu___he0703: 損保ジャパンの問題が色々流れてるけど間違いなく入らないほうがいいし相手にしたくない。

3月の事故、相手が確認不足、アクセル踏んだって言ってても損保ジャパンは避けれただろって。弁護士も呆れてるくらいだから。入らない方がいい。 https://t…


...

Analysis is done ignoring "\n".


定期!ニコ生やってますhttps://t.co/PJEjhVWpdeアニメ（CLANNAD、kanon、AIR、ＡＢ、氷菓、アクセルワールド、）、ゲーム、リトバス、クド、jubeat、ドラマニ、リフレクなどなどフォーローよろしくお願いします
おちんちんアクセル！！！！！動け！！！！動くんだ！！！！！！！動いた！！！！！！ギュギャギャッギュイーーーン！！！！！！！！ブゥオーーーーーーーン！！！！
折れた........
#shindanmaker
https://t.co/SiNVniBz2o


Analysis is done ignoring "\n".


9867 アクセル

ストップになる前にちょっと買えたので、ストップ高で売っちゃいます。
小銭稼ぎ。
まだ上がりそうな割安ですが、利益率低いのが気になるのでやめときます。


Analysis is done ignoring "\n".


In [142]:
fail_cnt

0

In [0]:
def cos_sim(v1, v2):
    try:
        v1 = torch.Tensor(v1)
        v2 = torch.Tensor(v2)
        
        return float(nn.CosineSimilarity(dim=0, eps=1e-6)(v1, v2))
    except Exception as e:
        print(e)
        return None

In [0]:
ref_sentence1_cos_sim = [cos_sim(vec, reference_sentence1_vec) for vec in tweet_vecs]
ref_sentence2_cos_sim = [cos_sim(vec, reference_sentence2_vec) for vec in tweet_vecs]
ref_sentence3_cos_sim = [cos_sim(vec, reference_sentence3_vec) for vec in tweet_vecs]

In [0]:
df_tweets = pd.DataFrame({
    'tweet': texts,
    'ref_sentence1_cos_sim': ref_sentence1_cos_sim,
    'ref_sentence2_cos_sim': ref_sentence2_cos_sim,
    'ref_sentence3_cos_sim': ref_sentence3_cos_sim,
})

In [146]:
df_tweets

,tweet,ref_sentence1_cos_sim,ref_sentence2_cos_sim,ref_sentence3_cos_sim
0,アクセル踏むとマッハの悪癖私もあるなあ……,0.469869,0.427910,0.579820
1,RT @008000dic: ナイス 【錆喰いビスコ4 業花の帝冠、花束の剣 (電撃文庫)/...,0.380300,0.370801,0.447865
2,RT @belmond_b_2434: やるわ\n【マリオカート８ DX】はい！ここでアクセ...,0.246965,0.231332,0.362356
3,Wﾚ*0ω0)おはよアクセル おはよロクサス(；∀； ≧＞,0.330730,0.343380,0.387070
4,アクセルナットすぐ取れる奴,0.490102,0.514295,0.519871
...,...,...,...,...
186,なんかいろんなこと考えるんだけどさ〜、擦り切れるまで頑張らなくていいと思うんだよね〜〜。アク...,0.245457,0.231704,0.418908
187,マニュアルトランスミッション(MT)ってのは、所謂自転車で言うギアのことだ。\n使えるなら使...,0.370742,0.364890,0.462792
188,定期!ニコ生やってますhttps://t.co/PJEjhVWpdeアニメ（CLANNAD、...,0.377564,0.376142,0.414742
189,おちんちんアクセル！！！！！動け！！！！動くんだ！！！！！！！動いた！！！！！！ギュギャギャ...,0.153178,0.139258,0.245268


In [147]:
reference_sentence1

'アクセルの社員がインサイダー取引により逮捕された。'

In [148]:
df_tweets.sort_values(by='ref_sentence1_cos_sim', ascending=False)

,tweet,ref_sentence1_cos_sim,ref_sentence2_cos_sim,ref_sentence3_cos_sim
182,アクセル・ワールド は2クールやぞ,0.555060,0.518501,0.539441
145,食事を即席麺などで済ませていたらアクセルに殴られる,0.550566,0.416368,0.659342
153,成功できない原因のほとんどは、アクセルとブレーキを同時に踏みしめている事である,0.548002,0.490881,0.577890
69,やはりアクセル飼おう,0.543704,0.505464,0.528053
79,シオンにちょっかいを出して泣かせた結果アクセルに顔面を殴られる,0.543510,0.409781,0.695974
...,...,...,...,...
106,おちんちんアクセル！！！！！動いてくれ！！！！動くんだ！！！！！！！動いた！！！！！！ギュギ...,0.142974,0.131422,0.239713
72,おちんちんアクセル！！！！！動け！！！！動けよ！！！！！！！動いた！！！！！！ギュインギュイ...,0.137847,0.124598,0.231539
131,おちんちんアクセル！！！！！可動しろ！！！！可動しろ！！！！！！！動いた！！！！！！ギュギャ...,0.137081,0.135700,0.225213
47,おちんちんアクセル！！！！！動いてくれ！！！！動けよ！！！！！！！動いた！！！！！！ギュオン...,0.129871,0.124537,0.222554


In [149]:
reference_sentence2

'アクセル、業績好調により上方修正を発表した。'

In [150]:
df_tweets.sort_values(by='ref_sentence2_cos_sim', ascending=False)

,tweet,ref_sentence1_cos_sim,ref_sentence2_cos_sim,ref_sentence3_cos_sim
19,6730アクセル 何？,0.524408,0.539281,0.471025
71,このトラックのアクセルON時の傾き加減ｗ,0.464797,0.519677,0.477539
105,@yui_ppppphm アクセルホッパーと空目した。笑,0.503542,0.518825,0.574432
182,アクセル・ワールド は2クールやぞ,0.555060,0.518501,0.539441
4,アクセルナットすぐ取れる奴,0.490102,0.514295,0.519871
...,...,...,...,...
53,おちんちんアクセル！！！！！動いてくれ！！！！動けよ！！！！！！！動いた！！！！！！ギュオン...,0.150639,0.133885,0.259288
106,おちんちんアクセル！！！！！動いてくれ！！！！動くんだ！！！！！！！動いた！！！！！！ギュギ...,0.142974,0.131422,0.239713
72,おちんちんアクセル！！！！！動け！！！！動けよ！！！！！！！動いた！！！！！！ギュインギュイ...,0.137847,0.124598,0.231539
47,おちんちんアクセル！！！！！動いてくれ！！！！動けよ！！！！！！！動いた！！！！！！ギュオン...,0.129871,0.124537,0.222554


In [151]:
reference_sentence3

'おじいちゃんがアクセルを踏み込みコンビニに突っ込んだ。'

In [152]:
df_tweets.sort_values(by='ref_sentence3_cos_sim', ascending=False)

,tweet,ref_sentence1_cos_sim,ref_sentence2_cos_sim,ref_sentence3_cos_sim
79,シオンにちょっかいを出して泣かせた結果アクセルに顔面を殴られる,0.543510,0.409781,0.695974
145,食事を即席麺などで済ませていたらアクセルに殴られる,0.550566,0.416368,0.659342
7,劣等生読み終わったら、バカテスとアクセルワールド読むことにした,0.522296,0.507174,0.637356
81,ブレーキとアクセルを踏み間違えるのは迷惑ポコ,0.520249,0.480547,0.631453
142,浅田真央ちゃんのトリプルアクセルで僕の菊の門をほじくってくださひぃぃぃいい！！おっほぉぉおお！！,0.425212,0.406753,0.625957
...,...,...,...,...
102,【続落から反発】(12/13 13:20)\n続落2日から反発：日本ＰＣサービス\n続落4日...,0.275406,0.286268,0.226937
131,おちんちんアクセル！！！！！可動しろ！！！！可動しろ！！！！！！！動いた！！！！！！ギュギャ...,0.137081,0.135700,0.225213
75,RT @kaburealnet: 【続落から反発】(12/13 13:20)\n続落2日から...,0.268088,0.284473,0.224586
47,おちんちんアクセル！！！！！動いてくれ！！！！動けよ！！！！！！！動いた！！！！！！ギュオン...,0.129871,0.124537,0.222554
